# Предварительная обработка данных(часть 1)
Коптев Данил Сергеевич

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as sts
import seaborn as sns
import math
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import Normalizer

**Часть 1**

1. Реализуем класс Norm для масштабирования количественных данных.

In [ ]:
class Norm :

  """ Класс TextSplitter используется для масштабирования количественных данных
    
    Attributes
    ----------
    method : str
        метода  масштабирования

    Methods
    -------
    fit(data)
        вычисляет необходимые статистики
    transform(data)
        выполняет преобразование выборки, переданной в качестве параметра, 
        используя значения статистик, полученные методом fit()
    fit_transform(data)
        выполняет  обе  операции  (вычисляет  необходимые статистики и 
        преобразует выборку, переданную в качестве параметра).
  """

  def __init__(self, method = "standardization"):
    self.method = method

  
  def fit(self, data, returnResult = False):
    """ Метод по обучающей выборке вычисляет необходимые статистики

        Parameters
        ----------
        data : str
            Переменная содержащая количественных данных
        returnResult: bool
            Параметр отвечающий за возрващение значений или нет
    """
    if self.method == "minmax":
      self.min = data.min(axis=0)
      self.max = data.max(axis=0)
      if  returnResult: 
        return {'min': self.min, 'max': self.max}
    elif self.method == "standardization":
      self.mean = np.mean(data, axis=0)
      self.std = np.std(data, axis=0)
      if  returnResult: 
        return {'mean': self.mean, 'std': self.std}
    elif self.method == "normalizer":
      self.vLen = np.sum(data**2, axis = 1)**0.5
      if  returnResult: 
        return self.vLen

  def transform(self, data):
    """ Метод выполняет преобразование выборки, переданной в качестве параметра, 
        используя значения статистик, полученные методом fit()

        Parameters
        ----------
        data : str
            Переменная содержащая количественных данных 
    """
    if self.method == "standardization":
      return (data-self.mean)/self.std
    elif self.method == "minmax":
      return (data-self.min)/(self.max-self.min)
    elif self.method == "normalizer":
      res = []
      for vectors in data:
        vLen = np.sum(vectors**2)**0.5
        res.append(vectors/vLen)
      return np.array(res)

  def fit_transform(self, data):
    """ Метод выполняет  обе  операции  (вычисляет  необходимые статистики и 
        преобразует выборку, переданную в качестве параметра).

        Parameters
        ----------
        data : str
            Переменная содержащая количественных данных 
    """
    if self.method == "standardization":
      mean = np.mean(data, axis=0)
      std = np.std(data, axis=0)
      return (data-mean)/std
    elif self.method == "minmax":
      min = data.min(axis=0)
      max = data.max(axis=0)
      return (data-min)/(max-min)
    elif self.method == "normalizer":
      res = []
      for vectors in data:
        vLen = np.sum(vectors**2)**0.5
        res.append(vectors/vLen)
      return np.array(res)

2. Протестируем класс Norm, используя сгенерированный набор данных

1) Создадим обучающую выборку, содержащую 20 объектов с 4 признаками, значения которых генерируются случайным образом и выведем размерность

In [ ]:
num_obj_train = 20
num_feat = 4
X_train = np.random.randint(-5, 5, size=(num_obj_train, num_feat))
norm = Norm('normalizer')
print(norm.fit(X_train, returnResult=True))
print("Размерность: ", num_obj_train, "x", num_feat)
print(X_train)

[2.23606798 7.14142843 4.69041576 3.74165739 6.4807407  6.78232998
 6.32455532 4.35889894 5.91607978 6.55743852 4.79583152 7.93725393
 4.24264069 8.18535277 6.164414   5.91607978 5.56776436 7.
 6.164414   6.164414  ]
Размерность:  20 x 4
[[ 0  0 -1 -2]
 [-5 -3  4  1]
 [-1  4 -2  1]
 [ 0 -1  2  3]
 [ 3 -4  1  4]
 [ 1 -5 -2  4]
 [-2 -4  4 -2]
 [ 1  1  4 -1]
 [ 1  3 -3  4]
 [-4 -3  3  3]
 [-3 -3 -2 -1]
 [-5 -2 -3 -5]
 [-2  3 -1  2]
 [ 4 -5 -5  1]
 [-5 -2 -3  0]
 [ 1  3  0 -5]
 [ 3 -3 -2  3]
 [-5 -4  2  2]
 [ 4  3 -2  3]
 [ 3 -4 -2  3]]


2) Аналогичным образом создадим тестовую выборку X_test, содержащую 10 объектов  и выведем размерность

In [ ]:
num_obj_test = 10
num_feat = 4
X_test = np.random.randint(-5, 5, size=(num_obj_test, num_feat))
print("Размерность: ", num_obj_train, "x", num_feat)
print(X_test)

Размерность:  20 x 4
[[-3 -5  3 -5]
 [ 1  4  4  1]
 [-2  0 -1  0]
 [ 2  2  2 -2]
 [-2  0 -1  0]
 [ 0  1  1  0]
 [ 3 -2  4  1]
 [ 1 -2 -5  2]
 [-4  0  2 -5]
 [ 4 -1 -3 -2]]


3) Найти средние значения и среднеквадратические отклонения, минимальное и максимальное  значения  длявсех  признаков,  а  также  длины векторов  объектов отдельно по обучающей и тестовой выборке 

In [ ]:
def descriptionSampling(X):
  print('Средние значения: ',  np.round(np.mean(X, axis = 0), 5))
  print('Среднеквадратические отклонения: ',  np.round(np.std(X, axis = 0),5))
  print('Минимальные значения признаков: ', np.min(X, axis = 0))
  print('Максимальные значения признаков: ', np.max(X, axis = 0))
  print('Длины векторов объектов: ', np.round(np.sum(X**2, axis = 1)**0.5, 5))

In [ ]:
print('По обучающей выборке:')
descriptionSampling(X_train)
print('\nПо тестовой выборке:')
descriptionSampling(X_test) 

По обучающей выборке:
Средние значения:  [-0.55 -1.3  -0.4   0.9 ]
Среднеквадратические отклонения:  [3.0899  2.98496 2.6533  2.71846]
Минимальные значения признаков:  [-5 -5 -5 -5]
Максимальные значения признаков:  [4 4 4 4]
Длины векторов объектов:  [2.23607 7.14143 4.69042 3.74166 6.48074 6.78233 6.32456 4.3589  5.91608
 6.55744 4.79583 7.93725 4.24264 8.18535 6.16441 5.91608 5.56776 7.
 6.16441 6.16441]

По тестовой выборке:
Средние значения:  [ 0.  -0.3  0.6 -1. ]
Среднеквадратические отклонения:  [2.52982 2.32594 2.87054 2.32379]
Минимальные значения признаков:  [-4 -5 -5 -5]
Максимальные значения признаков:  [4 4 4 2]
Длины векторов объектов:  [8.24621 5.83095 2.23607 4.      2.23607 1.41421 5.47723 5.83095 6.7082
 5.47723]


Среднее значения векторов обучающей выборке имеют диапазон между от -1.3 до 0.9

Среднее значения векторов тестовой выборке имеют диапазон между от -1 до 0.6, что не сильно отличается от среднего значения обучающей выборке

Среднеквадратические отклонения у обоих выборок сопаставимо и не имеет критических различий

Минимальные значения признаков у обоих выборок сопаставимо и не имеет критических различий

Максимальные значения признаков может различаться, но не критически

Длины векторов объектов имеют существенные различия

4) Для каждого метода масштабирования протестируем класс Norm по обучающей и тестовой выборке 

In [ ]:
# standardization -  метода  масштабирования стандартизации

print('По обучающей выборке:')

norm = Norm("standardization")
norm.fit(X_train)
standardization_X_test = norm.transform(X_test)
print(standardization_X_test)
descriptionSampling(standardization_X_test)

По обучающей выборке:
[[-0.79290517 -1.23954664  1.28142321 -2.17035008]
 [ 0.50163388  1.77556681  1.6583124   0.03678559]
 [-0.46927041  0.43551639 -0.22613351 -0.33107035]
 [ 0.82526865  1.1055416   0.90453403 -1.06678224]
 [-0.46927041  0.43551639 -0.22613351 -0.33107035]
 [ 0.17799912  0.77052899  0.52764485 -0.33107035]
 [ 1.14890341 -0.23450882  1.6583124   0.03678559]
 [ 0.50163388 -0.23450882 -1.73369023  0.40464154]
 [-1.11653994  0.43551639  0.90453403 -2.17035008]
 [ 1.47253818  0.10050378 -0.97991187 -1.06678224]]
Средние значения:  [ 0.178    0.33501  0.37689 -0.69893]
Среднеквадратические отклонения:  [0.81874 0.77922 1.08188 0.85482]
Минимальные значения признаков:  [-1.11653994 -1.23954664 -1.73369023 -2.17035008]
Максимальные значения признаков:  [1.47253818 1.77556681 1.6583124  0.40464154]
Длины векторов объектов:  [2.9185  2.48105 0.7554  1.96456 0.7554  1.00669 2.03134 1.86442 2.63912
 2.06802]


После применения стандартизации все признаки должно иметь среднее значение, равное 0 и дисперсию, равную 1, но как мы видим: 
* среднее значение у нас стримиться к 0, но не равняеется, а так же одно из значений имеет отрицательное значение, что говорит о успешном маштабирование не приходиться
* аналогичная ситуация с среднеквадратическим отклонением, так как оно стремиться к 1, а так же одно из значений превышает допустимое значение

Но не забываем о том, что не гарантируется получение каких-то конкретных минимальных и максимальных значений признаков

In [ ]:
# minmax -  метода масштабирования на отрезок
norm = Norm("minmax")
norm.fit(X_train)
minmax_X_test = norm.transform(X_test)
print(minmax_X_test)
descriptionSampling(minmax_X_test)

[[0.22222222 0.         0.88888889 0.        ]
 [0.66666667 1.         1.         0.66666667]
 [0.33333333 0.55555556 0.44444444 0.55555556]
 [0.77777778 0.77777778 0.77777778 0.33333333]
 [0.33333333 0.55555556 0.44444444 0.55555556]
 [0.55555556 0.66666667 0.66666667 0.55555556]
 [0.88888889 0.33333333 1.         0.66666667]
 [0.66666667 0.33333333 0.         0.77777778]
 [0.11111111 0.55555556 0.77777778 0.        ]
 [1.         0.44444444 0.22222222 0.33333333]]
Средние значения:  [0.55556 0.52222 0.62222 0.44444]
Среднеквадратические отклонения:  [0.28109 0.25844 0.31895 0.2582 ]
Минимальные значения признаков:  [0.11111111 0.         0.         0.        ]
Максимальные значения признаков:  [1.         1.         1.         0.77777778]
Длины векторов объектов:  [0.91625 1.69967 0.96225 1.38778 0.96225 1.22726 1.53156 1.07726 0.96225
 1.16534]


После применения масштабирование на отрезок ко всем признакам минимальное значение каждого признака станет равным нулю, а максимальное - единице, но что мы видим в нашем случае: 
*   Одно из минимальных значений признаков превышает допустимое значение
*   Одно из максимальных значений признаков меньше 1

Следовательно, что мы не можем говорить о успешном масштабированием на отрезок  [0, 1], но если мы взлянем на результат, то заметим, что значение не превышают допустимые значения



In [ ]:
# normalizer -  метода масштабирования нормализации
norm = Norm("normalizer")
# norm.fit(X_train) # ничего не делает
normalizer_X_test = norm.transform(X_test)
print(normalizer_X_test)
descriptionSampling(normalizer_X_test)

[[-0.36380344 -0.60633906  0.36380344 -0.60633906]
 [ 0.17149859  0.68599434  0.68599434  0.17149859]
 [-0.89442719  0.         -0.4472136   0.        ]
 [ 0.5         0.5         0.5        -0.5       ]
 [-0.89442719  0.         -0.4472136   0.        ]
 [ 0.          0.70710678  0.70710678  0.        ]
 [ 0.54772256 -0.36514837  0.73029674  0.18257419]
 [ 0.17149859 -0.34299717 -0.85749293  0.34299717]
 [-0.59628479  0.          0.2981424  -0.74535599]
 [ 0.73029674 -0.18257419 -0.54772256 -0.36514837]]
Средние значения:  [-0.06279  0.0396   0.09857 -0.15198]
Среднеквадратические отклонения:  [0.56486 0.43091 0.57561 0.35439]
Минимальные значения признаков:  [-0.89442719 -0.60633906 -0.85749293 -0.74535599]
Максимальные значения признаков:  [0.73029674 0.70710678 0.73029674 0.34299717]
Длины векторов объектов:  [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


Длины векторов объекта равны 1

Не наблюдается очевидных закономерностей в других значениях

Из-за того, что мы не можем натренировать с помощью метода fit и получить параметр длины вектора, то выборка нормализируется исключительно верно и это видно по значением, так как они проецируются на сферу радиуса 1

5) Применим к исходным данным встроенные обработчики библиотеки sklearn

In [ ]:
# StandardScaler - метода  масштабирования стандартизации библиотеки sklearn

standardScaler = StandardScaler()
standardScaler.fit(X_train)
standardScaler_X_test = standardScaler.transform(X_test)
print(standardScaler_X_test)
descriptionSampling(standardScaler_X_test)

[[-0.79290517 -1.23954664  1.28142321 -2.17035008]
 [ 0.50163388  1.77556681  1.6583124   0.03678559]
 [-0.46927041  0.43551639 -0.22613351 -0.33107035]
 [ 0.82526865  1.1055416   0.90453403 -1.06678224]
 [-0.46927041  0.43551639 -0.22613351 -0.33107035]
 [ 0.17799912  0.77052899  0.52764485 -0.33107035]
 [ 1.14890341 -0.23450882  1.6583124   0.03678559]
 [ 0.50163388 -0.23450882 -1.73369023  0.40464154]
 [-1.11653994  0.43551639  0.90453403 -2.17035008]
 [ 1.47253818  0.10050378 -0.97991187 -1.06678224]]
Средние значения:  [ 0.178    0.33501  0.37689 -0.69893]
Среднеквадратические отклонения:  [0.81874 0.77922 1.08188 0.85482]
Минимальные значения признаков:  [-1.11653994 -1.23954664 -1.73369023 -2.17035008]
Максимальные значения признаков:  [1.47253818 1.77556681 1.6583124  0.40464154]
Длины векторов объектов:  [2.9185  2.48105 0.7554  1.96456 0.7554  1.00669 2.03134 1.86442 2.63912
 2.06802]


Значения совпадают с результатом работы класса Norm('standardization')

Мы предерживаемся аналогичным выводам 

In [ ]:
# MinMaxScaler - метода масштабирования на отрезок библиотеки sklearn

minMaxScaler = MinMaxScaler()
minMaxScaler.fit(X_train)
minMaxScaler_X_test = minMaxScaler.transform(X_test)
print(minMaxScaler_X_test)
descriptionSampling(minMaxScaler_X_test)

[[0.22222222 0.         0.88888889 0.        ]
 [0.66666667 1.         1.         0.66666667]
 [0.33333333 0.55555556 0.44444444 0.55555556]
 [0.77777778 0.77777778 0.77777778 0.33333333]
 [0.33333333 0.55555556 0.44444444 0.55555556]
 [0.55555556 0.66666667 0.66666667 0.55555556]
 [0.88888889 0.33333333 1.         0.66666667]
 [0.66666667 0.33333333 0.         0.77777778]
 [0.11111111 0.55555556 0.77777778 0.        ]
 [1.         0.44444444 0.22222222 0.33333333]]
Средние значения:  [0.55556 0.52222 0.62222 0.44444]
Среднеквадратические отклонения:  [0.28109 0.25844 0.31895 0.2582 ]
Минимальные значения признаков:  [0.11111111 0.         0.         0.        ]
Максимальные значения признаков:  [1.         1.         1.         0.77777778]
Длины векторов объектов:  [0.91625 1.69967 0.96225 1.38778 0.96225 1.22726 1.53156 1.07726 0.96225
 1.16534]


Значения совпадают с результатом работы класса Norm('minmax')

Мы предерживаемся аналогичным выводам 

In [ ]:
# Normalizer - метода масштабирования нормализации библиотеки sklearn

normalizerScaler = Normalizer()
# normalizerScaler.fit(X_train) # ничего не делает 
normalizerScaler_X_test = normalizerScaler.transform(X_test)
print(normalizerScaler_X_test)
descriptionSampling(normalizerScaler_X_test)

[[-0.36380344 -0.60633906  0.36380344 -0.60633906]
 [ 0.17149859  0.68599434  0.68599434  0.17149859]
 [-0.89442719  0.         -0.4472136   0.        ]
 [ 0.5         0.5         0.5        -0.5       ]
 [-0.89442719  0.         -0.4472136   0.        ]
 [ 0.          0.70710678  0.70710678  0.        ]
 [ 0.54772256 -0.36514837  0.73029674  0.18257419]
 [ 0.17149859 -0.34299717 -0.85749293  0.34299717]
 [-0.59628479  0.          0.2981424  -0.74535599]
 [ 0.73029674 -0.18257419 -0.54772256 -0.36514837]]
Средние значения:  [-0.06279  0.0396   0.09857 -0.15198]
Среднеквадратические отклонения:  [0.56486 0.43091 0.57561 0.35439]
Минимальные значения признаков:  [-0.89442719 -0.60633906 -0.85749293 -0.74535599]
Максимальные значения признаков:  [0.73029674 0.70710678 0.73029674 0.34299717]
Длины векторов объектов:  [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


Значения совпадают с результатом работы класса Norm('normalizer')

Мы предерживаемся аналогичным выводам 

**Вывод**

На основание переденного отчета о работе выше мы делаем вывод, что тренировать записи на отличной от основной выборке не идеальное решение и мы можем наблюдать отличные результаты

Мы научились реализовывать классы на ЯП Python и поняли принцип работы методом масштабирования библиотеки sklearn